Project : CamemBERT  
Unit : Advanced Machine Learning   
MSc. Intelligent systems engineering  
SORBONNE UNIVERSITÉ  

--- Students ---  
@SSivanesan - Shivamshan SIVANESAN  
@Emirtas7 - Emir TAS  
@KishanthanKingston - Kishanthan KINGSTON  
code inspiré de :
- https://huggingface.co/blog/how-to-train
- https://github.com/theartificialguy/NLP-with-Deep-Learning/blob/master/BERT/Fine%20Tune%20BERT/fine_tuning_bert_with_MLM.ipynb
- https://github.com/mdabashar/Deep-Learning-Algorithms/blob/master/Retraining%20RoBERTa%20for%20MLM.ipynb

#### Roberta pré-entrainé sur une petite part de notre base de données

In [1]:
from transformers import RobertaTokenizerFast, RobertaForMaskedLM,AutoTokenizer 
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from utilis.utilis import PreProcessing
import torch
from torch.utils.data import Dataset

2023-12-10 13:19:11.021541: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 13:19:12.033517: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-10 13:19:12.033612: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-12-10 13:19:12.033624: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
file_path = 'fr_part_1_short.txt'
pre_process = PreProcessing(file_path)
input_texts = pre_process.read_dataset() 
# tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_len=512)
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [3]:

class MyDataset(Dataset):
    def __init__(self, texts, tokenizer, block_size:int = 128):
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        for text in texts:
            inputs = tokenizer(text, truncation=True, max_length=512, padding='max_length') 
            examples = {"input_ids": inputs.input_ids}
            grouped_examples = self.group_texts(examples, block_size)
            self.inputs.extend(grouped_examples["input_ids"])
            self.targets.extend(grouped_examples["labels"])

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        item = {"input_ids": self.inputs[idx], "labels": self.targets[idx]}
        return item

    def group_texts(self,examples, block_size):
        concatenated_examples = {k: examples[k] for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        if total_length >= block_size:
            total_length = (total_length // block_size) * block_size
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result

dataset = MyDataset(input_texts, tokenizer)

In [4]:
model = RobertaForMaskedLM.from_pretrained('roberta-base')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    output_dir="./roberta-mlm",
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=64,
    save_steps=10,
    save_total_limit=50,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [5]:
trainer_result = trainer.train()
trainer.save_model("./roberta-retrained")

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [10]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./roberta-retrained",
    tokenizer="roberta-base"
)
fill_mask("Je prépare un <mask> au chocolat.")

[{'score': 0.20697994530200958,
  'token': 4532,
  'token_str': ' maximum',
  'sequence': 'Je prépare un gateau au maximum.'},
 {'score': 0.06615154445171356,
  'token': 5059,
  'token_str': ' blog',
  'sequence': 'Je prépare un gateau au blog.'},
 {'score': 0.05863066762685776,
  'token': 1082,
  'token_str': ' site',
  'sequence': 'Je prépare un gateau au site.'},
 {'score': 0.03209709748625755,
  'token': 914,
  'token_str': ' match',
  'sequence': 'Je prépare un gateau au match.'},
 {'score': 0.027383187785744667,
  'token': 11739,
  'token_str': ' blend',
  'sequence': 'Je prépare un gateau au blend.'}]